# This Notebook enables you to call every single use API in the MTurk single use API catalog

### Prerequisites:
1. You have [an MTurk Requester account and linked AWS account](https://medium.com/@mechanicalturk/setting-up-your-amazon-mechanical-turk-mturk-requester-account-and-aws-account-a316e2f6a156)
2. You have [completed the account set up steps to call AmazonMechanicalTurk and AmazonMechanicalTurkCrowd](https://medium.com/@mechanicalturk/setting-up-your-amazon-mechanical-turk-and-aws-accounts-to-call-a-preview-api-59ade8beafd1)
3. You have configured a local AWS profile to call MTurk with the user you set up in step 2
4. You have [prepaid for HITs in your MTurk Requester account](https://requester.mturk.com/prepayments/new) to cover paying for Worker rewards (suggested amount to start is $5.00)

### If you haven't done so, install the mturk-crowd-beta Python client

In [ ]:
!pip install --upgrade mturk-crowd-beta-client --ignore-installed six

## Next we will import the packages we need to use the API
You'll need to do this before calling any API

In [3]:
from mturk_crowd_beta_client import MTurkCrowdClient
from boto3.session import Session
import uuid

## Next we set up a session

This examples assume you have a local AWS profile named __'mturk-crowd-caller'__, but you can authenticate however you like, including by directly passing in your access key and secret key.  The access key and secret key are for the User you created during setup with the policies AmazonMechanicalTurkFullAccess and AmazonMechanicalTurkCrowdFullAccess.

In [4]:
session = Session(profile_name='mturk-crowd-caller')
#session = Session(profile_name='mtf-caller')

crowd_client = MTurkCrowdClient(session)

## Available single use APIs:

### Natural Language Processing

* [sentiment-analysis](#sentiment-analysis)
* [emotion-detection](#emotion-detection)
* [semantic-similarity](#semantic-similarity)
* [collect-utterance-for-intent](#collect-utterance-for-intent)
* [intent-detection](#intent-detection)
* [text-categorization](#text-categorization)
* [named-entity-recognition](#named-entity-recognition)
* [coreference-resolution](#coreference-resolution)
* [key-phrase-extraction](#key-phrase-extraction)

### Computer Vision
* [image-contains](#image-contains)
* [image-categorization](#image-categorization)
* [image-similarity](#image-similarity)
* [bounding-box](#bounding-box)



## <a id='sentiment-analysis'>sentiment-analysis</a>

Input: `{ "text": "Everything is wonderful!" }`

Result: `{'sentiment': 'positive'}`

Max length of the input text is 400 characters.  Sentiment is one of positive, negative, neutral or cannot determine.  We ask up to 5 Workers until 2 of them agree on the answer.


When you create a Task using the sentiment-analysis API, you're automatically creating a Human Intelligence Task (HIT) on worker.mturk.com.  Here's an example of a sentiment analysis HIT.

![sentiment-analysis-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/sentiment-analysis-HIT-example.png)

### Create a Task

In [170]:
#set the function_name to the name of the single use API
function_name = 'sentiment-analysis'

In [77]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-0a21e7ff15df45df909d00812ef514da


In [78]:
# define the text that you want analyzed, up to 16k
text = 'The trip by @VP Pence was long planned. He is receiving great praise for leaving game after the players showed such disrespect for country!'

In [79]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 201, 'task': {'input': {'text': 'The trip by @VP Pence was long planned. He is receiving great praise for leaving game after the players showed such disrespect for country!'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-0a21e7ff15df45df909d00812ef514da'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [80]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'text': 'The trip by @VP Pence was long planned. He is receiving great praise for leaving game after the players showed such disrespect for country!'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-0a21e7ff15df45df909d00812ef514da'}}


### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [81]:
import pandas as pd

In [173]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('https://s3-us-west-2.amazonaws.com/mturk-sample-datasets/sentiment-analysis-example-inputs.csv')

In [174]:
# column 1 is the Task ID and column2 is the input data
data

,task_id,review_text
0,customer_review_1,I'll give it 3 stars only because they were on...
1,customer_review_2,I love these boots and these are my 2nd pair. ...
2,customer_review_3,I read many reviews regarding the brown ones a...


In [175]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, row.task_id,{'text': row.review_text})
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [ ]:
data

We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [84]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

In [85]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
#need to handle the case when not all tasks are in completed state
if 'processing' in data['state'].unique():
    data['sentiment'] = 'data still pending'    
else:
    data['sentiment'] = [r['result']['sentiment'] for r in responses]

In [ ]:
data

## <a id='image-contains'>image-contains</a>

Given an image and a type of object to look for, this API returns true or false.

Input: `{"image": {"url": "https://www.mturk.com/media/butterbean.jpg"}, "target": {"label": "dog"} }`

Result: `{"containsTarget": true}`

![image-contains-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/image-contains-HIT-example.jpeg)

In [56]:
function_name = 'image-contains-test'

### Create a Task

In [57]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-49377fbe8b764da3a4f2896fbcba23d7


In [58]:
# the URL of the image that you want annotated
image_url = 'https://urbanedge.blogs.rice.edu/files/2016/02/midtown-15wd4ck.jpg'

### The type of thing we're looking for

In [ ]:
label = 'Pedestrians'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                                   task_name,
                                   {'image': {'url': image_url}, 'target': {'label': label}})

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))        

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(
{'status_code': get_result.status_code, 'task': get_result.json()}))

### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [59]:
import pandas as pd

In [60]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('https://s3-us-west-2.amazonaws.com/mturk-sample-datasets/image-contains-example-inputs.csv')

In [61]:
# column 1 is the Task ID and column2 amd 3 are the input data
data

,task_id,image_url,Label
0,image-contains-1,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens satchel
1,image-contains-2,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens flip flops
2,image-contains-3,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens ballet flats
3,image-contains-4,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens wedge shoes
4,image-contains-5,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens flip flops
5,image-contains-6,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens wedge shoes
6,image-contains-7,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens mules
7,image-contains-8,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens mules
8,image-contains-9,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens ankle boots
9,image-contains-10,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens wedge shoes


In [62]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, 
                                       row.task_id,
                                       {'image': {'url': row.image_url},
                                        'target': {'label': row.Label} })
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [63]:
data

,task_id,image_url,Label,status_codes
0,image-contains-1,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens satchel,201
1,image-contains-2,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens flip flops,201
2,image-contains-3,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens ballet flats,201
3,image-contains-4,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens wedge shoes,201
4,image-contains-5,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens flip flops,201
5,image-contains-6,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens wedge shoes,201
6,image-contains-7,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens mules,201
7,image-contains-8,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens mules,201
8,image-contains-9,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens ankle boots,201
9,image-contains-10,https://s3-us-west-2.amazonaws.com/mturk-sampl...,womens wedge shoes,201


We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [67]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

In [165]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
#need to handle the case when not all tasks are in completed state
if 'processing' in data['state'].unique():
    data['containsTarget'] = 'data still pending'
else:
    data['containsTarget'] = [r['result']['containsTarget'] for r in responses]
    

In [ ]:
data

In [57]:
data.to_csv('image-contains-results.csv')

## <a id='emotion-detection'>emotion-detection</a>

This API determines the emotion of text.

Input:`{"text": "First time ever winning all three fantasy leagues AND @Seahawks win!"}`

Result: `{"emotion": "joy"}`

Emotion is one of Joy, Anger, Fear, Sadness, Surprise, Disgust, or Neutral.  Internal only: we ask up to 9 Workers to get agreement.

![emotion-detection-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/emotion-detection-HIT-example.png)

In [ ]:
function_name = 'emotion-detection'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'Just realised that I have #280characters so now I can finally tweet this: "Daenerys Stormborn of the House Targaryen, First of Her Name, the Unburnt, Queen of the Andals and the First Men, Khaleesi of the Great Grass Sea, Breaker of Chains, and Mother of Dragons'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=semantic-similarity>semantic-similarity</a>

Compare two text documents and rate them on how similar they are, on a scale between 0 and 1 where 1 is very similar.

Input: `{"text1": "The sky is blue.", "text2": "The sky was the color of blue."}`

Result: `{"similarityScore": 0.75}`

![semantic-similarity-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/semantic-similarity-HIT-example.jpeg)


In [106]:
function_name = 'semantic-similarity'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text1 = 'I\'m so hungry I could eat a horse'
text2 = 'I\'m hangry'

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text1': text1, 'text2': text2})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [107]:
import pandas as pd

In [108]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('https://s3-us-west-2.amazonaws.com/mturk-sample-datasets/semantic-similarity-example-inputs.csv')

In [109]:
# column 1 is the Task ID and column2 is the input data
data

,task_id,review1,review2
0,semantic-sim-1,"This is a nice portable DVD player with 7.5"" s...",I bought this DBPOWER DVD player for my son si...
1,semantic-sim-2,Nice product with easily adjustable bright LED...,This is exactly what I was looking for! The ad...
2,semantic-sim-3,I am thrilled to see this series make it to Bl...,I absolutely adore this series. I grew up with...
3,semantic-sim-4,"5.0 out of 5 starsWorks great, made of stainle...",3.0 out of 5 starsStainless Steel BBQ griller\...
4,semantic-sim-5,I initially gave this review 1 star because li...,This is a VERY dark turn for an already pretty...
5,semantic-sim-6,"While the book, at its heart, about writing ma...",I think this was a great book to add to any Ma...
6,semantic-sim-7,Quite nice update to the original book. Positi...,If you are like me and you do not like Heisig\...
7,semantic-sim-8,Just ordered two more sets of these. We\'ve us...,"Great product! The eggs come out perfect, it\'..."
8,semantic-sim-9,5.0 out of 5 starsFive Stars\r\nBydkuon Septem...,5.0 out of 5 starsUseful!\r\nByG Ortizon Octob...
9,semantic-sim-10,So far I am very happy with these pencils. The...,"Very smooth, crisp color if not turning them i..."


In [110]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, 
                                       row.task_id,
                                        {'text1': row.review1, 'text2': row.review2})
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [111]:
data

,task_id,review1,review2,status_codes
0,semantic-sim-1,"This is a nice portable DVD player with 7.5"" s...",I bought this DBPOWER DVD player for my son si...,201
1,semantic-sim-2,Nice product with easily adjustable bright LED...,This is exactly what I was looking for! The ad...,201
2,semantic-sim-3,I am thrilled to see this series make it to Bl...,I absolutely adore this series. I grew up with...,201
3,semantic-sim-4,"5.0 out of 5 starsWorks great, made of stainle...",3.0 out of 5 starsStainless Steel BBQ griller\...,201
4,semantic-sim-5,I initially gave this review 1 star because li...,This is a VERY dark turn for an already pretty...,201
5,semantic-sim-6,"While the book, at its heart, about writing ma...",I think this was a great book to add to any Ma...,201
6,semantic-sim-7,Quite nice update to the original book. Positi...,If you are like me and you do not like Heisig\...,201
7,semantic-sim-8,Just ordered two more sets of these. We\'ve us...,"Great product! The eggs come out perfect, it\'...",201
8,semantic-sim-9,5.0 out of 5 starsFive Stars\r\nBydkuon Septem...,5.0 out of 5 starsUseful!\r\nByG Ortizon Octob...,201
9,semantic-sim-10,So far I am very happy with these pencils. The...,"Very smooth, crisp color if not turning them i...",201


We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [143]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

In [163]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
#need to handle the case when not all tasks are in completed state
if 'processing' in data['state'].unique():
    data['result'] = [r['result'] for r in responses]
else:
    data['similarityScore'] = [r['result']['similarityScore'] for r in responses]

In [164]:
data

,task_id,review1,review2,status_codes,state,problem_details,similarityScore,result
0,semantic-sim-1,"This is a nice portable DVD player with 7.5"" s...",I bought this DBPOWER DVD player for my son si...,200,completed,None,{'similarityScore': 0.25},{'similarityScore': 0.25}
1,semantic-sim-2,Nice product with easily adjustable bright LED...,This is exactly what I was looking for! The ad...,200,completed,None,{'similarityScore': 0.75},{'similarityScore': 0.75}
2,semantic-sim-3,I am thrilled to see this series make it to Bl...,I absolutely adore this series. I grew up with...,200,completed,None,{'similarityScore': 0.75},{'similarityScore': 0.75}
3,semantic-sim-4,"5.0 out of 5 starsWorks great, made of stainle...",3.0 out of 5 starsStainless Steel BBQ griller\...,200,completed,None,{'similarityScore': 0.0},{'similarityScore': 0.0}
4,semantic-sim-5,I initially gave this review 1 star because li...,This is a VERY dark turn for an already pretty...,200,completed,None,{'similarityScore': 0.0},{'similarityScore': 0.0}
5,semantic-sim-6,"While the book, at its heart, about writing ma...",I think this was a great book to add to any Ma...,200,completed,None,{'similarityScore': 0.5},{'similarityScore': 0.5}
6,semantic-sim-7,Quite nice update to the original book. Positi...,If you are like me and you do not like Heisig\...,200,completed,None,{'similarityScore': 0.25},{'similarityScore': 0.25}
7,semantic-sim-8,Just ordered two more sets of these. We\'ve us...,"Great product! The eggs come out perfect, it\'...",200,completed,None,{'similarityScore': 0.75},{'similarityScore': 0.75}
8,semantic-sim-9,5.0 out of 5 starsFive Stars\r\nBydkuon Septem...,5.0 out of 5 starsUseful!\r\nByG Ortizon Octob...,200,processing,None,None,None
9,semantic-sim-10,So far I am very happy with these pencils. The...,"Very smooth, crisp color if not turning them i...",200,completed,None,{'similarityScore': 0.0},{'similarityScore': 0.0}


## <a id='collect-utterance-for-intent'>collect-utterance-for-intent</a>

Given a context and an intention, provide what you would say, in text, in that situation.

input: `{"context": "Someone recently bought a phone and it doesn't work", "intent": "They want to return a phone"}'`

result: `{'utterances': [{'text': 'The phone that I just bought stopped working.  I want to get a refund'}]}`



Here's an example of what the HIT might look like to Workers:
![collect-utterance-text-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/collect-utterance-text-HIT-example.png)

In [96]:
function_name = 'collect-utterance-for-intent'

### Create a Task

In [97]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-97823be23096400f8a9744de15d0308d


In [98]:
# provide the context and the intent
context = 'Someone is calling their doctor\'s office'
intent = 'I want to change an existing appointment'

In [99]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'context': context, 'intent': intent})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 201, 'task': {'input': {'context': "Someone is calling their doctor's office", 'intent': 'I want to change an existing appointment'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-97823be23096400f8a9744de15d0308d'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [100]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'context': "Someone is calling their doctor's office", 'intent': 'I want to change an existing appointment'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-97823be23096400f8a9744de15d0308d'}}


## <a id='image-categorization'>image-categorization</a>

Given an image URL and a list of categories, determine which category the image best belongs to.  You can optionally provide a description for each category to provide more guidance on what each category includes.


Input: 
```
{
  "input": {
    "image":{"url": "https://s3-us-west-2.amazonaws.com/...1m.png"},
    "categories": [ 
        {'label': 'Ankle Boot', 
           'description':  'description1' ,
           'positiveExamples': [
                {'image': {'url':'https://s3-...1d.jpg'}}
            ] ,
           'negativeExamples': [
                {'image': {'url':'https://s3-...1i.jpg'}}
            ] 
         },
         {'label': 'Wedge', 
           'description':  'description2 ',
           'positiveExamples': [
                {'image': {'url':'https://s3-...1l.png'}}
            ] ,
           'negativeExamples': [
                {'image': {'url':'https://s3-...1i.png'}}
            ] 
         },
     ]
   }
}
```

Result: `{"matches": [{"label":"Wedge"}]} `

![image-categorization-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/image-categorization-HIT-example.png)

In [ ]:
function_name = 'image-categorization'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# provide the context and the intent
image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1m.png'

label1 = 'Ankle Boot'
label2 = 'Knee High Boot'
label3 = 'Ballet Flats'
label4 = 'Flip Flops'
label5 = 'Sandal'
label6 = 'Mule'
label7 = 'Wedge'

# Descriptions amd example images are optional, but help guide the worker.
description1 = 'Closed toe shoe that covers entire foot up to ankle'
description2 = 'Closed toe shoe that covers entire foot and leg up to knee'
description3 = 'Closed toe shoe that covers foot and has no heel'
description4 = 'Open toe shoe that connects to foot through a single strap'
description5 = 'Open toe shoe that connects to foot through straps'
description6 = 'Closed toe shoe with no back on the heel of wear\'s foot'
description7 = 'Heeled shoe where sole is in the form of a wedge'
example1_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1d.jpg'
example2_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1z.jpg'
example3_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1dd.png'
example4_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1b.png'
example5_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1r.png'
example6_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1x.png'
example7_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1l.png'
example1b_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1i.jpg'
example2b_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1f.jpg'
example3b_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1y.png'
example4b_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1l.jpg'
example5b_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1q.jpg'
example6b_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1t.png'
example7b_image_url = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1i.png'




In [ ]:
# Create the task
put_result = crowd_client.put_task(function_name,
                                   task_name,
                                   {'image': {'url': image_url},
                                   'categories': [ {'label': label1,  'description':  description1 ,
                                                    'positiveExamples': [{'image': {'url': example1_image_url}}] ,
                                                    'negativeExamples': [{'image': {'url': example1b_image_url}}] },
                                                   {'label': label2,  'description':  description2,
                                                    'positiveExamples': [{'image': {'url': example2_image_url}}] ,
                                                    'negativeExamples': [{'image': {'url': example2b_image_url}}]  },
                                                   {'label': label3,  'description':  description3,
                                                    'positiveExamples': [{'image': {'url': example3_image_url}}] ,
                                                    'negativeExamples': [{'image': {'url': example3b_image_url}}]  },
                                                   {'label': label4,  'description':  description4,
                                                    'positiveExamples': [{'image': {'url': example4_image_url}}] ,
                                                    'negativeExamples': [{'image': {'url': example4b_image_url}}]  },
                                                   {'label': label5,  'description':  description5,
                                                      'positiveExamples': [{'image': {'url': example5_image_url}}] ,
                                                      'negativeExamples': [{'image': {'url': example5b_image_url}}]  },
                                                   {'label': label6,  'description':  description6,
                                                      'positiveExamples': [{'image': {'url': example6_image_url}}] ,
                                                      'negativeExamples': [{'image': {'url': example6b_image_url}}]  },
                                                   {'label': label7,  'description':  description7,
                                                    'positiveExamples': [{'image': {'url': example7_image_url}}] ,
                                                    'negativeExamples': [{'image': {'url': example7b_image_url}}]  }] })

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id='intent-detection'>intent-detection</a>

Given a list of intentions, categorize the best intention that matches the text given.

input: 
```
{
  "input": {
     "text": "my son was stung by bees and I need to know if I need to go to the ER",
     "intents":[
                 {"label": "Schedule an appointment", 
                  "description": "Caller expresses an intent to visit with health care professional"
                  "positiveExamples":  [
                        {"text": "I need to make an appointment with Dr. Smith" },
                        {"text": "Can the doctor see me today?"}], 
                  "negativeExamples":  [
                        {"text": "Do you sell burritos?" },
                        {"text": "Do you accept credit cards?"}] },
                 {"label": "Medical Record Request", 
                  "description": "Caller expresses a desire to obtain copies of their medical history"
                  "positiveExamples":  [
                        {"text": "I need a copy of my kids’ immunization records" },
                        {"text": "Can you fax my test results to my surgeon"}], 
                  "negativeExamples":  [
                       {"text": "I need to refill my prescription"},
                        {"text": "Do you have my test results?"}] }]
   }
}
```

result:`{"matches": [{"label": "Schedule an appointment"}]}`

Here's an example of what the task might look like for Workers.
![text-intent-detection-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/text-intent-detection-HIT-example.png)

In [101]:
function_name = 'intent-detection'

### Create a Task

In [102]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-5bb6b000030947b9a56b7821ae181a1a


In [103]:
# define the text to be analysed and a list of intentions to categorize the text for
text = 'my son was stung by bees and I need to know if I need to go to the ER'
intents =  [{'label': 'Schedule an appointment', 'description': 'e.g. I need to make an appointment with Dr. Smith'}, {'label': 'Medical Record Request', 'description': 'e.g. I need a copy of my kids immunization records'}]

In [104]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                                   task_name,
                                  {'text': text,
                                  'intents': intents})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 201, 'task': {'input': {'intents': [{'description': 'e.g. I need to make an appointment with Dr. Smith', 'label': 'Schedule an appointment'}, {'description': 'e.g. I need a copy of my kids immunization records', 'label': 'Medical Record Request'}], 'text': 'my son was stung by bees and I need to know if I need to go to the ER'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-5bb6b000030947b9a56b7821ae181a1a'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [105]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'intents': [{'description': 'e.g. I need to make an appointment with Dr. Smith', 'label': 'Schedule an appointment'}, {'description': 'e.g. I need a copy of my kids immunization records', 'label': 'Medical Record Request'}], 'text': 'my son was stung by bees and I need to know if I need to go to the ER'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-5bb6b000030947b9a56b7821ae181a1a'}}


## <a id=text-categorization>text-categorization</a>

Given a piece of text and a list of categories, choose the best cateogry that fits the text.

input: 
```
{
  "input": {
    "text": "These are great. They do run a touch small. 
           I almost could go a half size up from my normal size." , 
    "description": "Customer mentions appearance or style of item.',
    "categories": [ 
         {"label": "Style", 
          "description":  "Customer mentions appearance or
                             style of item." ,
          "positiveExamples": [
               {"text": "Not the color I ordered"},
               {"text": "No longer in season"} 
           ] ,
          "negativeExamples": [
               {"text": "Too large"},
               {"text": "Fabric was too thin"}
           ] 
         },
         {"label": "Fit", 
          "description":  "Customer mentions the cut or
                            fit of item." ,
          "positiveExamples": [
               {"text": "This was too short."},
               {"text": "Doesn't work for my body type."} 
           ] ,
          "negativeExamples": [
               {"text": "I don't like the baggy look"},
               {"text": "Neckline is too low cut"}
           ] 
         }
      ]
   }
}
```

result: `{"matches": [{"label" : "Fit"}] }`

Here's an example of a HIT to MTurk Workers generated by calling this API
![text-categorization-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/text-categorization-HIT-example.png)

In [ ]:
function_name = 'text-categorization'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text to be analysed and a list of categories to assess for
text = 'These are great. They do run a touch small. I almost could go a half size up from my normal size.'
categories =  [{'label': 'style', 'description': 'related to the look of the product'}, {'label': 'fit', 'description': 'related to the sizing or how it fits'}, {'label': 'quality', 'description': 'related to how well made the product is'}, {'label': 'price', 'description': 'related to cost or value of the product'}]

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                                   task_name,
                                  {'text': text,
                                  'categories': categories})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=named-entity-recognition>named-entity-recognition</a>

Given a paragraph of text, label the span of text that represent entities of various categories, such as person and location.

input: `{'text': 'Harry Potter is midway through his training as a wizard and his coming of age. Harry wants to get away from the pernicious Dursleys and go to the International Quidditch Cup. He wants to find out about the mysterious event that's supposed to take place at Hogwarts this year, an event involving two other rival schools of magic, and a competition that hasn't happened for a hundred years. He wants to be a normal, fourteen-year-old wizard. But unfortunately for Harry Potter, he's not normal - even by wizarding standards. And in his case, different can be deadly.'}`


result: `{'entities': [{'span': {'endIndex': 12, 'startIndex': 0, 'text': 'Harry Potter', 'type': 'person'}}, {'span': {'endIndex': 84, 'startIndex': 79, 'text': 'Harry', 'type': 'person'}}, {'span': {'endIndex': 131, 'startIndex': 123, 'text': 'Dursleys', 'type': 'person'} }, {'span': {'endIndex': 77, 'startIndex': 60, 'text': 'his coming of age', 'type': 'date'} }, {'span': {'endIndex': 173, 'startIndex': 146, 'text': 'International Quidditch Cup', 'type': 'event'}}, {'span': {'endIndex': 222, 'startIndex': 217, 'text': 'event', 'type': 'event'}}, {'span': {'endIndex': 284, 'startIndex': 279, 'text': 'event', 'type': 'event'} }, {'span': {'endIndex': 346, 'startIndex': 335, 'text': 'competition', 'type': 'event'} }, {'span': {'endIndex': 264, 'startIndex': 256, 'text': 'Hogwarts', 'type': 'organization'} }, {'span': {'endIndex': 318, 'startIndex': 311, 'text': 'schools', 'type': 'organization'} }, {'span': {'endIndex': 264, 'startIndex': 256, 'text': 'Hogwarts', 'type': 'organization'}}]}`

![NER-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/ner-HIT-example.png)

### Create a Task

In [87]:
#set the function_name to the name of the single use API
function_name = 'named-entity-recognition'

In [88]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-5161f8ef447b4742896ec97597084862


In [89]:
# define the text that you want analyzed, up to 16k
text = 'Harry Potter is midway through his training as a wizard and his coming of age. Harry wants to get away from the pernicious Dursleys and go to the International Quidditch Cup. He wants to find out about the mysterious event that\'s supposed to take place at Hogwarts this year, an event involving two other rival schools of magic, and a competition that hasn\'t happened for a hundred years.'

In [84]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 201, 'task': {'input': {'text': "Harry Potter is midway through his training as a wizard and his coming of age. Harry wants to get away from the pernicious Dursleys and go to the International Quidditch Cup. He wants to find out about the mysterious event that's supposed to take place at Hogwarts this year, an event involving two other rival schools of magic, and a competition that hasn't happened for a hundred years."}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-8157aa6957f94628b7c31788a18d6ff0'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [85]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'text': "Harry Potter is midway through his training as a wizard and his coming of age. Harry wants to get away from the pernicious Dursleys and go to the International Quidditch Cup. He wants to find out about the mysterious event that's supposed to take place at Hogwarts this year, an event involving two other rival schools of magic, and a competition that hasn't happened for a hundred years."}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-8157aa6957f94628b7c31788a18d6ff0'}}


### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [90]:
import pandas as pd

In [92]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('https://s3-us-west-2.amazonaws.com/mturk-sample-datasets/named-entity-recognition-inputs.csv')
#data = pd.read_csv('named-entity-recognition-inputs.csv')

In [93]:
# column 1 is the Task ID and column2 is the input data
data

,task_id,review_text
0,customer_review_1,"I generally use Pampers Sensitive baby wipes, ..."
1,customer_review_2,I ordered this toilet paper thinking it was co...
2,customer_review_3,"Excellent, solid built cable with compact head..."
3,customer_review_4,And finally: Samsung\'s Gear Fit Pro fitness t...
4,customer_review_5,I purchased these Towbows and Copic Markers an...
5,customer_review_6,Bring your photographic subjects closer with t...
6,customer_review_7,FCC Chairman Newton Minow termed television a ...
7,customer_review_8,The Grammys nominated The Chainsmokers for Bes...
8,customer_review_9,We were recommended this restaurant by our ric...
9,customer_review_10,Found this Hotel on Trip Advisor. Excellent va...


In [94]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, row.task_id,{'text': row.review_text})
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [95]:
data

,task_id,review_text,status_codes
0,customer_review_1,"I generally use Pampers Sensitive baby wipes, ...",200
1,customer_review_2,I ordered this toilet paper thinking it was co...,200
2,customer_review_3,"Excellent, solid built cable with compact head...",200
3,customer_review_4,And finally: Samsung\'s Gear Fit Pro fitness t...,400
4,customer_review_5,I purchased these Towbows and Copic Markers an...,200
5,customer_review_6,Bring your photographic subjects closer with t...,200
6,customer_review_7,FCC Chairman Newton Minow termed television a ...,200
7,customer_review_8,The Grammys nominated The Chainsmokers for Bes...,200
8,customer_review_9,We were recommended this restaurant by our ric...,200
9,customer_review_10,Found this Hotel on Trip Advisor. Excellent va...,200


We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [100]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

In [167]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
#need to handle the case when not all tasks are in completed state
#data['result'] = [r['result'] for r in responses]
if 'processing' in data['state'].unique():
    data['result'] = 'data still pending'    
else:
    data['result'] = [r['result'] for r in responses]


In [ ]:
data

In [31]:
#write results to csv
data.to_csv('NER-results-sample.csv')

## <a id='bounding-box'> bounding-box </a>

Given an image, draw a box around particular objects, specified by labels.  This API returns the coordinates of labeled objects in the image.

input: `{"image": {"url": "https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg"}, "labels": [{"label": "car"}, {"label": "pedestrians"}]}`

result: `{'annotations': [{'boundingBox': {'height': 55, 'left': 896, 'top': 665, 'width': 51}, 'label': 'car'}, {'boundingBox': {'height': 50, 'left': 555, 'top': 515, 'width': 63}, 'label': 'car'}, {'boundingBox': {'height': 51, 'left': 539, 'top': 590, 'width': 58}, 'label': 'car'}, {'boundingBox': {'height': 44, 'left': 216, 'top': 512, 'width': 55}, 'label': 'car'}, {'boundingBox': {'height': 54, 'left': 873, 'top': 475, 'width': 53}, 'label': 'car'}, {'boundingBox': {'height': 56, 'left': 1083, 'top': 490, 'width': 47}, 'label': 'car'}, {'boundingBox': {'height': 42, 'left': 164, 'top': 659, 'width': 66}, 'label': 'car'}, {'boundingBox': {'height': 46, 'left': 524, 'top': 213, 'width': 50}, 'label': 'car'}, {'boundingBox': {'height': 48, 'left': 814, 'top': 311, 'width': 53}, 'label': 'car'}]}`

This API currently only supports upright bounding boxes.  

Here's an example of what the Worker sees:
![bound-box-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/bound-box-HIT-example.png)

In [8]:
function_name = 'bounding-box'

### Create a Task

In [9]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-52162080f2964bbeafc7a1b3e42dc78e


In [10]:
# provide the context and the intent
image_url = 'https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg'
labels =  [{"label": "car"}, {"label": "pedestrians"}]

In [11]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                            {'image': {'url': image_url},
                            'labels': labels})

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 201, 'task': {'input': {'image': {'url': 'https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg'}, 'labels': [{'label': 'car'}, {'label': 'pedestrians'}]}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-52162080f2964bbeafc7a1b3e42dc78e'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [12]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'image': {'url': 'https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg'}, 'labels': [{'label': 'car'}, {'label': 'pedestrians'}]}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-52162080f2964bbeafc7a1b3e42dc78e'}}


## <a id=image-similarity>image-similarity</a>
This API takes in two images, specified by URL, and returns a score for how similar those two images are.  

input: `{'input': {'image1': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1a.png'}, 'image2': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1b.png'}}`

result: `{'similarityScore': 0.75}`

Here's an example of a HIT that's shown to Workers:
![image-similarity-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/image-similarity-HIT-example.png)

In [18]:
function_name = 'image-similarity'

### Create a Task

In [19]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-847ac56e049249228283881984914ebb


In [20]:
# the URLs of the images that you want to compare
image_url_1 = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1a.png'
image_url_2 = 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1b.png'

In [21]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,task_name,
                                   {'image1': {'url': image_url_1},
                                    'image2': {'url': image_url_2}})

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))        

PUT response: {'status_code': 201, 'task': {'input': {'image1': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1a.png'}, 'image2': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1b.png'}}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-847ac56e049249228283881984914ebb'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [22]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'image1': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1a.png'}, 'image2': {'url': 'https://s3-us-west-2.amazonaws.com/mturk-sample-media/images-to-compare/image-similarity-1b.png'}}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-847ac56e049249228283881984914ebb'}}


## <a id=coreference-resolution>coreference-resolution</a>

Coreference resolution is the task of finding all expressions that refer to the same entity in a text. This API returns references and the position of the start and end of each reference.

input: `{ "text": "Megan said "I voted for Hilary because she better aligns with my values"" }`

result:  `{'coreferences': {'references': {'ref-1': [{'endIndex': 6, 'startIndex': 0, 'text': 'Megan ', 'type': 'ref-1'}, {'endIndex': 14, 'startIndex': 12, 'text': 'I ', 'type': 'ref-1'}, {'endIndex': 64, 'startIndex': 62, 'text': 'my', 'type': 'ref-1'}], 'ref-2': [{'endIndex': 30, 'startIndex': 24, 'text': 'Hilary', 'type': 'ref-2'}, {'endIndex': 42, 'startIndex': 39, 'text': 'she', 'type': 'ref-2'}]}}}`



Here's an example of a HIT that's shown to Workers:
![co-reference-resolution-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/co-reference-resolution-HIT-example.png)

### Create a Task

In [76]:
#set the function_name to the name of the single use API
function_name = 'coreference-resolution'

In [77]:
# automatically generate a random task ID
#task_name = 'my-test-task-' + uuid.uuid4().hex
task_name = 'my-test-task-656e8384878a40d59f55289a48085d0d'
print(task_name)

my-test-task-656e8384878a40d59f55289a48085d0d


In [78]:
# define the text that you want analyzed, up to 16k
text = 'Megan said \"I voted for Hilary because she better aligns with my values\"'

In [79]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 201, 'task': {'input': {'text': 'Megan said "I voted for Hilary because she better aligns with my values"'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-656e8384878a40d59f55289a48085d0d'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [80]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'text': 'Megan said "I voted for Hilary because she better aligns with my values"'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-656e8384878a40d59f55289a48085d0d'}}


## <a id=keyphrase-extraction>key-phrase-extraction</a> 

Key phrases provide a concise description of a document’s content; they are useful for document categorization, clustering, indexing, search, and summarization; quantifying semantic similarity with other documents; as well as conceptualizing particular knowledge domains.  Key phrase extraction is the process of extracting the important and topic words in a document.

input: `{ "text": "This is the second Boots I brought from this brand, the material they are used to make these boots are quite decent, it’s not real leather though, but once you get it on hand you can tell that it is definitely high quality material, the last boots I brought cost me 45 bucks, it lasted about 1 year and half, very remarkable durability I got to say, beside this the interior is super soft and very comfortable to wear. If you are looking for mid-range priced boots with high quality this is the best shot for you." }`

result: `{'keyPhrases': [{'endIndex': 25, 'startIndex': 19, 'text': 'Boots '}, {'endIndex': 115, 'startIndex': 103, 'text': 'quite decent'}, {'endIndex': 138, 'startIndex': 122, 'text': 'not real leather'}, {'endIndex': 231, 'startIndex': 210, 'text': 'high quality material'}, {'endIndex': 307, 'startIndex': 292, 'text': '1 year and half'}, {'endIndex': 335, 'startIndex': 309, 'text': 'very remarkable durability'}, {'endIndex': 388, 'startIndex': 378, 'text': 'super soft'}, {'endIndex': 409, 'startIndex': 393, 'text': 'very comfortable'}, {'endIndex': 458, 'startIndex': 442, 'text': 'mid-range priced'}, {'endIndex': 482, 'startIndex': 470, 'text': 'high quality'}]}`



Here's an example of a HIT that's shown to Workers: 
![key-phrase-extraction-HIT-example](https://s3-us-west-2.amazonaws.com/mturk-sample-tasks/key-phrase-extraction-HIT-example.png)

### Create a Task

In [ ]:
#set the function_name to the name of the single use API
function_name = 'key-phrase-extraction'

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'This is the second Boots I brought from this brand, the material they are used to make these boots are quite decent, it’s not real leather though, but once you get it on hand you can tell that it is definitely high quality material, the last boots I brought cost me 45 bucks, it lasted about 1 year and half, very remarkable durability I got to say, beside this the interior is super soft and very comfortable to wear. If you are looking for mid-range priced boots with high quality this is the best shot for you.'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))